# Regularized polynomial regression

In [72]:
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

class RegularizedLinearRegression:
    def __init__(self, num_features):
        # Min and max of the interval where random values for the weights should be sampled
        b = -1.
        a = 1.
        # Weights vector includes slopes and bias
        self.w = (b - a) * np.random.random_sample((num_features + 1, 1)) + a

    # `lam` is the regularization factor
    def fit(self, X, y, learning_rate, epochs, lam=0.):
        self.loss_history = []

        N = X.shape[0]
        
        for _ in range(epochs):
            dlossdw = (2/N) * X.T @ (self.predict(X) - y)
            # Do not include bias in regularization term
            dlossdw[1:] += 2 * lam * self.w[1:]
            self.w -= learning_rate * dlossdw
            training_prediction = self.predict(X)
            loss = self.loss(training_prediction, y, lam)
            self.loss_history.append(loss)
    
    def loss(self, prediction, y, lam):
        N = len(prediction)
        return np.linalg.norm(prediction - y)**2 / N + lam * np.sum(self.w[1:]**2)

    def predict(self, X):
        return X @ self.w

# Generate synthetic data for a quartic model
np.random.seed(0)
X = np.linspace(-3, 3, 30).reshape(-1, 1)
# Quartic ground truth model with noise
y = 3 + 1.5 * X - 2.0 * X**2 + 0.3 * X**3 - 0.1 * X**4 + np.random.normal(0, 1, X.shape)

# Create polynomial features up to degree 12
degree = 12
X_poly = np.hstack([X**i for i in range(degree + 1)])

# Add a column of ones for the bias term
X_poly = np.hstack([np.ones((X_poly.shape[0], 1)), X_poly]) 

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

# Standardize the features (excluding the bias term) and labels
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# Standardize all columns except the first (bias term)
X_train[:, 1:] = scaler_X.fit_transform(X_train[:, 1:])
X_test[:, 1:] = scaler_X.transform(X_test[:, 1:])

# Standardize labels
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))

In [ ]:
# Create and train the model with L2 regularization
model = RegularizedLinearRegression(num_features=X_train.shape[1] - 1)
model.fit(X_train, y_train_scaled, learning_rate=0.01, epochs=1000, lam=0.01)

# Make predictions on training and test sets
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Inverse transform predictions for y back to original scale
y_train_pred = scaler_y.inverse_transform(y_train_pred)
y_test_pred = scaler_y.inverse_transform(y_test_pred)

# Calculate R-squared scores using sklearn's r2_score
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
print("Training R-squared score:", train_r2)
print("Test R-squared score:", test_r2)

# Plot the results
plt.scatter(X, y, color='blue', label='Original data')
X_poly_all = np.hstack([np.ones((X_poly.shape[0], 1)), scaler_X.transform(X_poly[:, 1:])])  # Standardize for plotting
y_pred_all = model.predict(X_poly_all)
y_pred_all = scaler_y.inverse_transform(y_pred_all)
plt.plot(X, y_pred_all, color='red', label=f'Polynomial regression (degree={degree})')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Quartic Polynomial Regression with L2 Regularization')
plt.legend()
plt.show()

In [ ]:
# Create and train the model without regularization
model = RegularizedLinearRegression(num_features=X_train.shape[1] - 1)
model.fit(X_train, y_train_scaled, learning_rate=0.01, epochs=1000, lam=0.)

# Make predictions on training and test sets
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Inverse transform predictions for y back to original scale
y_train_pred = scaler_y.inverse_transform(y_train_pred)
y_test_pred = scaler_y.inverse_transform(y_test_pred)

# Calculate R-squared scores using sklearn's r2_score
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
print("Training R-squared score:", train_r2)
print("Test R-squared score:", test_r2)

# Plot the results
plt.scatter(X, y, color='blue', label='Original data')
X_poly_all = np.hstack([np.ones((X_poly.shape[0], 1)), scaler_X.transform(X_poly[:, 1:])])  # Standardize for plotting
y_pred_all = model.predict(X_poly_all)
y_pred_all = scaler_y.inverse_transform(y_pred_all)
plt.plot(X, y_pred_all, color='red', label=f'Polynomial regression (degree={degree})')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Quartic Polynomial Regression with L2 Regularization')
plt.legend()
plt.show()